In [64]:
import tkinter as tk
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

In [134]:
# Button Callbacks
def add_clicked():
    # Add ellipse to plot
    pass
    
def rem_clicked(): 
    # Remove last plotted ellipse
    pass

def see_clicked(): 
    # See
    pass
    
def calc_rad_clicked(): 
    # Calc
    pass

def calc_rad_inv_clicked(): 
    # Calc
    pass

# Intensidad: Varía entre -1 y 1, ya que es aditiva. (I)
# Inclinación: Ángulo en grados de rotación respecto al eje x. (A)
# Semi-eje X: Valor del semi-eje X de la elipse, entre (0 ; 1]. (X)
# Semi-eje Y: Valor del semi-eje Y de la elipse, entre (0 ; 1]. (Y)
# Centro X: Coordenada X del centro de la elipse, entre [-1 ; 1]. (CX)
# Centro Y: Coordenada Y del centro de la elipse, entre [-1 ; 1]. (CY)

In [128]:
# Figures
f_1 = Figure(figsize=(2,2), dpi=100)
phantom_fig = f_1.add_subplot(111)
phantom_fig.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

f_2 = Figure(figsize=(3,3), dpi=100)
rad_fig = f_2.add_subplot(111)
rad_fig.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

f_3 = Figure(figsize=(3,3), dpi=100)
rad_inv_fig = f_3.add_subplot(111)
rad_inv_fig.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

In [136]:
# Create root window 
root = Tk() 
  
# Root window title and dimension 
root.title("Tomography Simulator") 
root.geometry('1080x640') 

# Widgets 

# Label Frames 

# Phantom
label_frame_phantom = LabelFrame(root, text = 'Phantom') 
label_frame_phantom.pack(expand = 'yes', fill = 'both', side = tk.LEFT)
label_frame_add = LabelFrame(label_frame_phantom, text = 'Add ellipse') 
label_frame_add.pack()
  
Label(label_frame_add, text = 'Intensity').grid(column=0, row=0)  
Label(label_frame_add, text = 'Inclination').grid(column=0, row=1)
Label(label_frame_add, text = 'Semi-axis X').grid(column=0, row=2)
Label(label_frame_add, text = 'Semi-axis Y').grid(column=0, row=3)
Label(label_frame_add, text = 'Center X').grid(column=0, row=4)
Label(label_frame_add, text = 'Center Y').grid(column=0, row=5)

intensity = Entry(label_frame_add, width=10) 
intensity.grid(column=1, row=0)
inc = Entry(label_frame_add, width=10) 
inc.grid(column=1, row=1)
axis_x = Entry(label_frame_add, width=10) 
axis_x.grid(column=1, row=2)
axis_y = Entry(label_frame_add, width=10) 
axis_y.grid(column=1, row=3)
center_x = Entry(label_frame_add, width=10) 
center_x.grid(column=1, row=4)
center_y = Entry(label_frame_add, width=10) 
center_y.grid(column=1, row=5)

# Radon Frame
label_frame_radon = LabelFrame(root, text = 'Radon Transform') 
label_frame_radon.pack(expand = 'yes', fill = 'both', side = tk.RIGHT)

label_frame_rad = LabelFrame(label_frame_radon, text = 'Radon') 
label_frame_rad.pack(expand = 'yes', fill = 'both', side = tk.TOP)

label_frame_rad_inv = LabelFrame(label_frame_rad, text = 'Radon Inverse') 
label_frame_rad_inv.pack(expand = 'yes', fill = 'both', side = tk.BOTTOM)


# Radon
label_frame_r_config = LabelFrame(label_frame_rad, text = 'Config') 
label_frame_r_config.pack(side = tk.LEFT)

Label(label_frame_r_config, text = 'From').grid(column=0, row=0)
Label(label_frame_r_config, text = 'Step').grid(column=0, row=1)
Label(label_frame_r_config, text = 'To').grid(column=0, row=2)
Label(label_frame_r_config, text = 'Angle').grid(column=0, row=4)

fro = Entry(label_frame_r_config, width=10) 
fro.grid(column=1, row=0)
step = Entry(label_frame_r_config, width=10) 
step.grid(column=1, row=1)
to = Entry(label_frame_r_config, width=10) 
to.grid(column=1, row=2)
angle = Entry(label_frame_r_config, width=10) 
angle.grid(column=1, row=4)

# Radon Inv
label_frame_ri_config = LabelFrame(label_frame_rad_inv, text = 'Config') 
label_frame_ri_config.pack(side = tk.LEFT)

Label(label_frame_ri_config, text = 'From').grid(column=0, row=0)
Label(label_frame_ri_config, text = 'Step').grid(column=0, row=1)
Label(label_frame_ri_config, text = 'To').grid(column=0, row=2)
Label(label_frame_ri_config, text = 'Interpolation').grid(column=0, row=3)
Label(label_frame_ri_config, text = 'Filter').grid(column=0, row=4)

fro_i = Entry(label_frame_ri_config, width=10) 
fro_i.grid(column=1, row=0)
step_i = Entry(label_frame_ri_config, width=10) 
step_i.grid(column=1, row=1)
to_i = Entry(label_frame_ri_config, width=10) 
to_i.grid(column=1, row=2)

var_int = StringVar(label_frame_ri_config)
var_int.set("No selection") # default value
var_filt = StringVar(label_frame_ri_config)
var_filt.set("No Selection") # default value

interp = OptionMenu(label_frame_ri_config, var_int, "one", "two", "three")
interp.grid(column=1, row=3)
interp = OptionMenu(label_frame_ri_config, var_filt, "one", "two", "three")
interp.grid(column=1, row=4)

# Canvas
canvas_p = FigureCanvasTkAgg(f_1, label_frame_phantom)
canvas_p.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

canvas_r = FigureCanvasTkAgg(f_2, label_frame_rad)
canvas_r.get_tk_widget().pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

canvas_ri = FigureCanvasTkAgg(f_3, label_frame_rad_inv)
canvas_ri.get_tk_widget().pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Button widgets
btn_add = Button(label_frame_add, text = "Add", fg = "red", command=add_clicked)
btn_rem = Button(label_frame_add, text = "Remove Last", fg = "red", command=rem_clicked)
  
btn_add.grid(column=0, row=6)
btn_rem.grid(column=1, row=6)

btn_see = Button(label_frame_r_config, text = "See", fg = "red", command=see_clicked)
btn_calc = Button(label_frame_r_config, text = "Calculate", fg = "red", command=calc_rad_clicked)

btn_calc.grid(column=1, row=3)
btn_see.grid(column=1, row=5)

btn_calc_inv = Button(label_frame_ri_config, text = "Calculate", fg = "red", command=calc_rad_inv_clicked)
btn_calc_inv.grid(column=1, row=5)

# Run Mainloop
root.mainloop()